# Liouville Space Propagation of Optimized Universal Set of Gates

In [1]:
import QDYN
from select_for_prop import prepare_prop
from os.path import join
from notebook_utils import get_Liouville_error
import pandas as pd
from collections import OrderedDict
from generate_zeta_universal import write_config
from run_oct import propagate
from QDYN.pulse import Pulse, pulse_tgrid
from mgplottools.io import writetotxt
from QDYN.linalg import vectorize

In [2]:
cluster_args = '--cluster-ini=cluster.ini --parallel=38 --jobs=1'

## Propagate optimized pulses in Liouville Space

In [3]:
universal_root = './runs_zeta_detailed/w2_5900MHz_wc_6200MHz'
prop_root = './propagate_universal'
universal_rf = {
  'H_L': (universal_root+'/50ns_w_center_H_left',     prop_root+'/rho/H_L'),
  'H_R': (universal_root+'/50ns_w_center_H_right',    prop_root+'/rho/H_R'),
  'S_L': (universal_root+'/50ns_w_center_Ph_left',    prop_root+'/rho/S_L'),
  'S_R': (universal_root+'/50ns_w_center_Ph_right',   prop_root+'/rho/S_R'),
  'PE':  (universal_root+'/PE_LI_BGATE_50ns_cont_SM', prop_root+'/rho/BGATE')
}

In [4]:
for name in universal_rf:
    oct_folder = universal_rf[name][0]
    prop_folder = universal_rf[name][1]
    prepare_prop(oct_folder, prop_folder, dry_run=False)

In [5]:
! ./run_prop.py {cluster_args} --rwa --threads --rho --stage-folder='rho' {prop_root}

See tabular overview below

## Propagate optimized pulses in Liouville Space, weak dissipation

In [6]:
universal_root = './runs_zeta_detailed/w2_5900MHz_wc_6200MHz'
prop_root = './propagate_universal'
universal_rf = {
  'H_L': (universal_root+'/50ns_w_center_H_left',     prop_root+'/rho_weak/H_L'),
  'H_R': (universal_root+'/50ns_w_center_H_right',    prop_root+'/rho_weak/H_R'),
  'S_L': (universal_root+'/50ns_w_center_Ph_left',    prop_root+'/rho_weak/S_L'),
  'S_R': (universal_root+'/50ns_w_center_Ph_right',   prop_root+'/rho_weak/S_R'),
  'PE':  (universal_root+'/PE_LI_BGATE_50ns_cont_SM', prop_root+'/rho_weak/BGATE')
}

In [7]:
for name in universal_rf:
    oct_folder = universal_rf[name][0]
    prop_folder = universal_rf[name][1]
    prepare_prop(oct_folder, prop_folder, dry_run=False)

In [8]:
for name in universal_rf:
    prop_folder = universal_rf[name][1]
    ! perl -p -i -e 's/kappa\s*=\s*0.05_MHz/kappa   = 0.005_MHz/' {prop_folder}/config
    ! perl -p -i -e 's/gamma_1\s*=\s*0.012_MHz/gamma_1 = 0.0012_MHz/' {prop_folder}/config
    ! perl -p -i -e 's/gamma_2\s*=\s*0.012_MHz/gamma_2 = 0.0012_MHz/' {prop_folder}/config

In [9]:
! ./run_prop.py {cluster_args} --rwa --rho --threads --stage-folder='rho_weak' {prop_root}

In [10]:
names = []; errs = []; errs_weak = []
for name in universal_rf:
    names.append(name)
    rf_weak = universal_rf[name][1]
    rf = rf_weak.replace('_weak', '')
    errs.append(get_Liouville_error(join(rf, 'prop.log')))
    errs_weak.append(get_Liouville_error(join(rf_weak, 'prop.log')))
err_table = pd.DataFrame(OrderedDict(
    [('normal', errs), ('weak', errs_weak)]), index=names)
print(err_table.to_string(float_format=lambda f:'%.2e'%f));

      normal     weak
H_L 4.46e-03 1.01e-03
S_R 4.07e-03 7.39e-04
H_R 5.03e-03 1.33e-03
S_L 4.79e-03 1.29e-03
PE  4.70e-03 1.06e-03


## Propagate field-free Hamiltonian in Hilbert space, w/o dissipation

The gate errors compare to the theoretical limit *at the parameter point*. We can determine this limit by propagating the field-free Hamiltonian in Hilbert space without dissipation. Then, we propagate the field-free Hamiltonian in Liouville space, with dissipation. The error with which the Liouville space propagation implements the unitary gate obtained in Hilbert space is the theoretical limit.

In [11]:
wc = 6.2 # GHZ
w2 = 5.9 # GHz
w1 = 6.00 # GHz

In [12]:
field_free_gates = {}

In [13]:
for T in [10, 20, 50, 100, 200]:
    nt = T * 20
    rf = './propagate_universal/hilbert_nodiss_ff_%d' % T; QDYN.shutil.mkdir(rf)
    write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
    pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
    pulse.write(join(rf, 'pulse.guess'))
    pulse.write(join(rf, 'pulse.dat'))
    U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True, dissipation=False)
    writetotxt(join(rf, 'target_gate.dat'), vectorize(U.real), vectorize(U.imag))
    field_free_gates[T] = U

## Propagate field-free Hamiltonian in Liouville space

The following determins the limit at the specific parameter point

In [14]:
for T in [10, 20, 50, 100, 200]:
    nt = T * 20
    rf_hilbert = './propagate_universal/hilbert_nodiss_ff_%d' % T
    rf = './propagate_universal/liouville_ff/%03d' % T; QDYN.shutil.mkdir(rf)
    O = field_free_gates[T]
    writetotxt(join(rf, 'target_gate.dat'), vectorize(O.real), vectorize(O.imag))
    prepare_prop(rf_hilbert, rf, dry_run=False)
    ! perl -p -i -e 's/gate = CPHASE/gate = target_gate.dat/' {rf}/config

In [15]:
! ./run_prop.py {cluster_args} --rwa --rho --threads --stage-folder=liouville_ff propagate_universal

In [16]:
err_0_B = {}
for T in [10, 20, 50, 100, 200]:
    rf = './propagate_universal/liouville_ff/%03d' % T
    err_0_B[T] = get_Liouville_error(join(rf, 'prop.log'))

In [17]:
for T in [10, 20, 50, 100, 200]:
    print "%3d ns: %.2e" % (T, err_0_B[T])

 10 ns: 7.23e-04
 20 ns: 1.45e-03
 50 ns: 3.61e-03
100 ns: 7.20e-03
200 ns: 1.43e-02


In [18]:
for T in [10, 20, 50, 100, 200]:
    nt = T * 20
    rf_hilbert = './propagate_universal/hilbert_nodiss_ff_%d' % T
    rf = './propagate_universal/liouville_weak_ff/%03d' % T; QDYN.shutil.mkdir(rf)
    O = field_free_gates[T]
    writetotxt(join(rf, 'target_gate.dat'), vectorize(O.real), vectorize(O.imag))
    prepare_prop(rf_hilbert, rf, dry_run=False)
    ! perl -p -i -e 's/gate = CPHASE/gate = target_gate.dat/' {rf}/config
    ! perl -p -i -e 's/kappa\s*=\s*0.05_MHz/kappa   = 0.005_MHz/' {rf}/config
    ! perl -p -i -e 's/gamma_1\s*=\s*0.012_MHz/gamma_1 = 0.0012_MHz/' {rf}/config
    ! perl -p -i -e 's/gamma_2\s*=\s*0.012_MHz/gamma_2 = 0.0012_MHz/' {rf}/config

In [19]:
! ./run_prop.py {cluster_args} --rwa --rho --threads --stage-folder=liouville_weak_ff propagate_universal

In [20]:
err_0_B_weak = {}
for T in [10, 20, 50, 100, 200]:
    rf = './propagate_universal/liouville_weak_ff/%03d' % T
    err_0_B_weak[T] = get_Liouville_error(join(rf, 'prop.log'))

In [21]:
for T in [10, 20, 50, 100, 200]:
    print "%3d ns: %.2e" % (T, err_0_B_weak[T])

 10 ns: 7.24e-05
 20 ns: 1.45e-04
 50 ns: 3.62e-04
100 ns: 7.23e-04
200 ns: 1.45e-03
